In [14]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

In [15]:
# userid, psuedo item, rating (3.0 - 5.0)
# rating_20core_df = pd.read_csv("./rating_5000_cls_hc_20core_iterative.csv", sep='\t', names=['userid','pseudoitemid','rating'])
rating_20core_df = pd.read_csv("./rating_5000_cls_hc_10core_iterative.csv", sep='\t', names=['userid','pseudoitemid','rating'])
rating_20core_df.head()

,userid,pseudoitemid,rating
0,00043ad3146f033d4bf1beb502fa6e09,41,3.0
1,00043ad3146f033d4bf1beb502fa6e09,60,3.0
2,00043ad3146f033d4bf1beb502fa6e09,175,3.0
3,00043ad3146f033d4bf1beb502fa6e09,191,3.0
4,00043ad3146f033d4bf1beb502fa6e09,265,3.0


In [16]:
rating_20core_df.shape

(110371, 3)

In [17]:
np.unique(rating_20core_df['userid']).shape

(4005,)

In [18]:
np.unique(rating_20core_df['pseudoitemid']).shape

(2673,)

In [19]:
print np.unique(rating_20core_df['userid']).shape

(4005,)


In [20]:
print np.unique(rating_20core_df['pseudoitemid']).shape

(2673,)


In [27]:
loan_features_df = pd.read_csv('./hac_features_with_clst_assignment.csv', sep='\t')
loan_features_df.head()

,LOAN_ID,LOAN_AMOUNT,PFR,ACTIVITY_Agriculture,ACTIVITY_Air Conditioning,ACTIVITY_Animal Sales,ACTIVITY_Aquaculture,ACTIVITY_Arts,ACTIVITY_Auto Repair,ACTIVITY_Bakery,...,COUNTRY_Zimbabwe,700_cls,1000_cls,1500_cls,2000_cls,3000_cls,5000_cls,7000_cls,10000_cls,20000_cls
0,1420951.0,500.0,12.500000,0,0,0,0,0,0,0,...,0,281,140,140,1158,1158,578,1633,4938,2468
1,1412008.0,275.0,5.263158,0,0,0,0,0,0,0,...,0,253,253,126,1960,1960,979,6628,1656,3226
2,1439600.0,1200.0,33.333333,0,0,0,0,0,0,0,...,0,578,578,578,288,143,588,5785,5785,5785
3,1435212.0,2150.0,33.333333,0,0,0,0,0,0,0,...,0,55,141,106,90,2061,1030,5434,5434,10869
4,1440703.0,750.0,7.142857,0,0,0,0,0,0,0,...,0,87,65,24,135,2858,2858,5717,2858,11327


In [28]:
#initially
len(set(loan_features_df['5000_cls']))

5000

In [29]:
loan_features_df.columns.values.shape

(229,)

### turn everything into categorical variables, even the numerical ones!

In [31]:
loan_features_df.drop(columns=['700_cls', '1000_cls', '1500_cls', '2000_cls', '3000_cls',
                               '7000_cls', '10000_cls', '20000_cls'], axis=1, inplace=True)
loan_features_df = loan_features_df[loan_features_df['5000_cls'].isin(rating_20core_df['pseudoitemid'])]
loan_features_df.rename(columns={'5000_cls': 'pseudoitemid'}, inplace=True)
loan_features_df.head()

/home/nsonboli/.conda/envs/ipykernel_py2/lib/python2.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,LOAN_ID,LOAN_AMOUNT,PFR,ACTIVITY_Agriculture,ACTIVITY_Air Conditioning,ACTIVITY_Animal Sales,ACTIVITY_Aquaculture,ACTIVITY_Arts,ACTIVITY_Auto Repair,ACTIVITY_Bakery,...,COUNTRY_Togo,COUNTRY_Turkey,COUNTRY_Uganda,COUNTRY_Ukraine,COUNTRY_United States,COUNTRY_Vietnam,COUNTRY_Yemen,COUNTRY_Zambia,COUNTRY_Zimbabwe,pseudoitemid
0,1420951.0,500.0,12.500000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,578
1,1412008.0,275.0,5.263158,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,979
2,1439600.0,1200.0,33.333333,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,588
3,1435212.0,2150.0,33.333333,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1030
4,1440703.0,750.0,7.142857,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2858


In [32]:
len(set(loan_features_df['pseudoitemid']))

2673

In [42]:
newDf = []
for ps_itemid, itsdf in loan_features_df.groupby('pseudoitemid'):
#     print(ps_itemid, itsdf)
    itsdf.drop(['LOAN_ID', 'pseudoitemid'], axis=1, inplace=True)
    avg_df = itsdf.mean(axis=0)
    avg_df['pseudoitemid'] = int(ps_itemid)
    newDf.append(pd.DataFrame(avg_df).T)
    
l_f_df = pd.concat(newDf, ignore_index=True) 
l_f_df.head()

(0,          LOAN_ID  LOAN_AMOUNT        PFR  ACTIVITY_Agriculture  \
10145  1387083.0        500.0   6.666667                     0   
17365  1375327.0        500.0  25.000000                     0   
21411  1330495.0        400.0   4.545455                     0   
26825  1333987.0        200.0  12.500000                     0   
38627  1273929.0        275.0  33.333333                     0   
39653  1273972.0        200.0  20.000000                     0   

       ACTIVITY_Air Conditioning  ACTIVITY_Animal Sales  ACTIVITY_Aquaculture  \
10145                          0                      0                     0   
17365                          0                      0                     0   
21411                          0                      0                     0   
26825                          0                      0                     0   
38627                          0                      0                     0   
39653                          0               

(9,          LOAN_ID  LOAN_AMOUNT        PFR  ACTIVITY_Agriculture  \
3723   1425746.0        350.0  16.666667                     0   
7439   1420606.0        425.0  25.000000                     0   
8673   1430133.0        675.0  20.000000                     0   
9424   1440480.0        350.0  33.333333                     0   
10369  1389415.0        350.0  25.000000                     0   
22316  1348141.0        250.0  33.333333                     0   
23606  1336138.0        325.0  33.333333                     0   
24709  1320896.0        350.0  33.333333                     0   
25599  1338446.0        500.0  33.333333                     0   
25627  1317296.0        350.0  16.666667                     0   
37258  1289164.0        350.0  33.333333                     0   
38907  1288075.0        525.0  20.000000                     0   

       ACTIVITY_Air Conditioning  ACTIVITY_Animal Sales  ACTIVITY_Aquaculture  \
3723                           0                      0 

(17,          LOAN_ID  LOAN_AMOUNT        PFR  ACTIVITY_Agriculture  \
13651  1353337.0       1000.0  12.500000                     0   
14614  1369241.0       1025.0  20.000000                     0   
18988  1340952.0       1550.0   6.666667                     0   
22031  1350002.0       1025.0  10.000000                     0   
23532  1324455.0       1500.0  33.333333                     0   
28621  1310146.0       2000.0   8.333333                     0   
31800  1292431.0       1000.0  25.000000                     0   

       ACTIVITY_Air Conditioning  ACTIVITY_Animal Sales  ACTIVITY_Aquaculture  \
13651                          0                      0                     0   
14614                          0                      0                     0   
18988                          0                      0                     0   
22031                          0                      0                     0   
23532                          0                      0      

(27,          LOAN_ID  LOAN_AMOUNT        PFR  ACTIVITY_Agriculture  \
456    1421185.0        700.0  25.000000                     0   
14615  1369246.0        575.0  16.666667                     0   
15510  1358910.0        700.0   5.555556                     0   
15685  1369237.0        575.0  20.000000                     0   
18505  1334661.0        700.0   5.555556                     0   
31017  1294177.0        250.0  10.000000                     0   
33355  1278664.0        250.0   5.000000                     0   
33494  1285342.0        475.0  12.500000                     0   
34202  1288861.0        250.0  10.000000                     0   
36803  1269361.0        250.0   7.142857                     0   
38793  1292135.0        250.0  25.000000                     0   

       ACTIVITY_Air Conditioning  ACTIVITY_Animal Sales  ACTIVITY_Aquaculture  \
456                            0                      0                     0   
14615                          0        

(34,          LOAN_ID  LOAN_AMOUNT        PFR  ACTIVITY_Agriculture  \
3054   1409601.0       1000.0   7.692308                     1   
4115   1423627.0       1500.0   4.545455                     1   
4592   1434961.0       1200.0  11.111111                     1   
4593   1434967.0       1800.0  12.500000                     1   
4614   1428728.0       1500.0   5.882353                     1   
5451   1433157.0       1000.0  10.000000                     1   
5670   1426047.0       1975.0   8.333333                     1   
6342   1436314.0       1000.0  10.000000                     1   
6914   1436220.0       2300.0  11.111111                     1   
7098   1410317.0       1000.0   7.142857                     1   
8527   1428790.0       1000.0   8.333333                     1   
8529   1428793.0       1500.0   7.692308                     1   
8831   1427588.0       1500.0  14.285714                     1   
9736   1394929.0       1600.0   6.666667                     1   
9744 

...

(66,          LOAN_ID  LOAN_AMOUNT        PFR  ACTIVITY_Agriculture  \
1050   1428481.0        300.0   5.882353                     0   
8995   1397292.0        425.0   7.692308                     0   
9641   1421311.0        575.0  25.000000                     0   
20187  1374923.0       1125.0  14.285714                     0   
28740  1326639.0       1675.0   5.882353                     0   
32210  1293131.0        425.0   7.142857                     0   
32508  1297627.0        275.0  25.000000                     0   
33237  1286610.0        700.0   4.545455                     0   

       ACTIVITY_Air Conditioning  ACTIVITY_Animal Sales  ACTIVITY_Aquaculture  \
1050                           0                      0                     0   
8995                           0                      0                     0   
9641                           0                      0                     0   
20187                          0                      0                     

(74,          LOAN_ID  LOAN_AMOUNT        PFR  ACTIVITY_Agriculture  \
729    1407078.0       4125.0  33.333333                     0   
10218  1390921.0       4350.0  33.333333                     0   
10494  1390967.0       2725.0  33.333333                     0   
10973  1390969.0       3950.0  33.333333                     0   
13747  1376076.0       5450.0  33.333333                     0   
14454  1368638.0       5525.0  33.333333                     0   
14740  1376024.0       4575.0  25.000000                     0   
14742  1376021.0       2375.0  25.000000                     0   
14743  1376023.0       3075.0  25.000000                     0   
15528  1376072.0       1450.0  25.000000                     0   
15632  1356396.0       2800.0  33.333333                     0   
16770  1377028.0       3275.0  33.333333                     0   
16965  1376183.0       5225.0  25.000000                     0   
17707  1366749.0       1550.0  33.333333                     0   
18427

(81,          LOAN_ID  LOAN_AMOUNT        PFR  ACTIVITY_Agriculture  \
610    1376186.0      10000.0   4.761905                     1   
2755   1401165.0      10000.0   4.761905                     1   
3258   1413619.0       4000.0   9.090909                     1   
3560   1399361.0      25000.0   9.090909                     1   
3562   1428758.0       7000.0  16.666667                     1   
6818   1400027.0      10000.0  11.111111                     1   
8008   1415292.0       7000.0  20.000000                     1   
8213   1426504.0      10000.0   6.666667                     1   
10903  1362629.0       4000.0  16.666667                     1   
13543  1343100.0       4000.0  25.000000                     1   
15826  1327069.0       5000.0   8.333333                     1   
18696  1361040.0      10000.0   7.692308                     1   
24183  1316022.0       3000.0   7.142857                     1   
24216  1310535.0      10000.0   4.761905                     1   
25070

(89,          LOAN_ID  LOAN_AMOUNT        PFR  ACTIVITY_Agriculture  \
656    1408071.0        650.0  14.285714                     0   
798    1419872.0        825.0  14.285714                     0   
1103   1420695.0        525.0  16.666667                     0   
2630   1428080.0       1150.0  20.000000                     0   
5061   1409146.0        525.0  16.666667                     0   
5742   1439032.0       1050.0  20.000000                     0   
7220   1421127.0        475.0  14.285714                     0   
7772   1441733.0        900.0  25.000000                     0   
8928   1419968.0        775.0  25.000000                     0   
11309  1401236.0        900.0  25.000000                     0   
11926  1399186.0        400.0  20.000000                     0   
12274  1401543.0        900.0  25.000000                     0   
13104  1399395.0        475.0  20.000000                     0   
13117  1399176.0        400.0  16.666667                     0   
13908

KeyboardInterrupt: 

In [34]:
cols = l_f_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
l_f_df = l_f_df[cols]
l_f_df.head()

,pseudoitemid,LOAN_AMOUNT,PFR,ACTIVITY_Agriculture,ACTIVITY_Air Conditioning,ACTIVITY_Animal Sales,ACTIVITY_Aquaculture,ACTIVITY_Arts,ACTIVITY_Auto Repair,ACTIVITY_Bakery,...,COUNTRY_Timor-Leste,COUNTRY_Togo,COUNTRY_Turkey,COUNTRY_Uganda,COUNTRY_Ukraine,COUNTRY_United States,COUNTRY_Vietnam,COUNTRY_Yemen,COUNTRY_Zambia,COUNTRY_Zimbabwe
0,0.0,345.833333,17.007576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,362.500000,15.935020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,1050.000000,14.102564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,1430.882353,11.707453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,1095.833333,18.148148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Bucketing loan amount and funding rate

In [35]:
# l_f_df['pfr_buck'] = pd.qcut(l_f_df['PFR'], 4, labels=['pfr_1', 'pfr_2', 'pfr_3', 'pfr_4'])
l_f_df['pfr_buck'] = pd.qcut(l_f_df['PFR'], 4, labels=False)
l_f_df['pfr_buck'].value_counts()

0    669
3    668
1    668
2    668
Name: pfr_buck, dtype: int64

In [36]:
# l_f_df['loan_buck'] = pd.qcut(l_f_df['LOAN_AMOUNT'], 10, labels=['loan_1','loan_2','loan_3','loan_4','loan_5',
#                                                                     'loan_6','loan_7','loan_8','loan_9','loan_10'])

l_f_df['loan_buck'] = pd.qcut(l_f_df['LOAN_AMOUNT'], 10, labels=False)
l_f_df['loan_buck'].value_counts()

0    274
4    273
3    269
9    268
8    267
7    266
1    265
6    265
5    263
2    263
Name: loan_buck, dtype: int64

In [37]:
l_f_df.drop(columns=['LOAN_AMOUNT', 'PFR'], axis=1, inplace=True)
l_f_df.head()

,pseudoitemid,ACTIVITY_Agriculture,ACTIVITY_Air Conditioning,ACTIVITY_Animal Sales,ACTIVITY_Aquaculture,ACTIVITY_Arts,ACTIVITY_Auto Repair,ACTIVITY_Bakery,ACTIVITY_Barber Shop,ACTIVITY_Beauty Salon,...,COUNTRY_Turkey,COUNTRY_Uganda,COUNTRY_Ukraine,COUNTRY_United States,COUNTRY_Vietnam,COUNTRY_Yemen,COUNTRY_Zambia,COUNTRY_Zimbabwe,pfr_buck,loan_buck
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,5
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,5


In [38]:
l_f_df = pd.get_dummies(l_f_df, columns=['loan_buck', 'pfr_buck'])
l_f_df.head()

,pseudoitemid,ACTIVITY_Agriculture,ACTIVITY_Air Conditioning,ACTIVITY_Animal Sales,ACTIVITY_Aquaculture,ACTIVITY_Arts,ACTIVITY_Auto Repair,ACTIVITY_Bakery,ACTIVITY_Barber Shop,ACTIVITY_Beauty Salon,...,loan_buck_4,loan_buck_5,loan_buck_6,loan_buck_7,loan_buck_8,loan_buck_9,pfr_buck_0,pfr_buck_1,pfr_buck_2,pfr_buck_3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,1,0,0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0


In [40]:
np.unique(l_f_df['pseudoitemid']).shape

(2673,)

In [302]:
# l_f_df.to_csv('./loan_feature_matrix_hc.csv',index=False)

In [303]:
# l_f_df.to_csv('/home/nsonboli/opp_fairness/data/Kiva/loan_feature_matrix_hc_20core.csv', index=False)
l_f_df.to_csv('/home/nsonboli/opp_fairness/data/Kiva/loan_feature_matrix_hc_10core.csv', index=False)

In [304]:
l_f_df.head()

,pseudoitemid,ACTIVITY_Agriculture,ACTIVITY_Air Conditioning,ACTIVITY_Animal Sales,ACTIVITY_Aquaculture,ACTIVITY_Arts,ACTIVITY_Auto Repair,ACTIVITY_Bakery,ACTIVITY_Barber Shop,ACTIVITY_Beauty Salon,...,loan_buck_4,loan_buck_5,loan_buck_6,loan_buck_7,loan_buck_8,loan_buck_9,pfr_buck_0,pfr_buck_1,pfr_buck_2,pfr_buck_3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,1,0,0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0


In [305]:
x = l_f_df.stack()
blist = []

for i in range(l_f_df.shape[0]):
    alist = []
    for k in range(x[i].shape[0]):
        if x[i].values[k] != 0:
            if (x[i].keys()[k] == 'pseudoitemid'):
                itemid = x[i].values[k]
            else:
                alist.append([itemid, x[i].keys()[k]])
    blist.extend(alist)

blist_df = pd.DataFrame(blist, columns=['itemid', 'feature'])
# blist_df.to_csv('/home/nsonboli/opp_fairness/data/Kiva/loan_feature_df_hc_20core.csv', index=False)
blist_df.to_csv('/home/nsonboli/opp_fairness/data/Kiva/loan_feature_df_hc_10core.csv', index=False)

In [306]:
blist_df.head()

,itemid,feature
0,4997.0,ACTIVITY_Fruits & Vegetables
1,4997.0,GENDER_M
2,4997.0,COUNTRY_Kenya
3,4997.0,loan_buck_0
4,4997.0,pfr_buck_2
